## Export Blue Carbon tiffs

* #### **Step 1:** Take Blue carbon Asset from GEE and export to the cloud storage as tiffs
* #### **Step 2:** Take exported tiffs and unify into one single file and reupload to GCS

### Step 1: Export Asset (GEE) to tiff (GCS)

In [14]:
import ee

In [15]:
ee.Authenticate()

Enter verification code:  4/1AX4XfWhGuoWoLpdQhia-S586lcmJloRadD9L2KEbEnlsDZ726T4NcU9paNc



Successfully saved authorization token.


In [16]:
ee.Initialize()

In [17]:
region = ee.Geometry.Polygon([-180, 40, 0,40,180, 40,180,-40,0,-40,-180,-40])
coll = ee.ImageCollection("projects/global-mangrove-watch/mangrove-properties/mangrove_total_co2e_1996--2016").select('total_co2e')
collectionList = coll.toList(coll.size())
collectionSize = collectionList.size().getInfo()

In [22]:
i= 0
img = ee.Image(collectionList.get(i))
img_name = ee.String(img.get("system:id")).split("/").get(-1).getInfo()
year = img_name[-4:]

params = {
'image':img,
'description':'toc_'+year,
'bucket':'magrove_atlas',
'fileNamePrefix':'ee_export_tiffs/total_organic_carbon/toc_'+year+'/toc_'+year, 
'fileFormat':'GeoTIFF',
'region': region.getInfo()['coordinates'], 
'skipEmptyTiles': False,
'maxPixels': 1e12,
'scale':30
}

task = ee.batch.Export.image.toCloudStorage(**params)
task.start()

In [58]:
for i in range(collectionSize):
    img = ee.Image(collectionList.get(i))
    img_name = ee.String(img.get("system:id")).split("/").get(-1).getInfo()
    year = img_name[-4:]

    params = {
    'image':img,
    'description':'toc_'+year,
    'bucket':'magrove_atlas',
    'fileNamePrefix':'ee_export_tiffs/total_organic_carbon/toc_'+year+'/toc_'+year, 
    'fileFormat':'GeoTIFF',
    'region': region.getInfo()['coordinates'], 
    'skipEmptyTiles': True,
    'maxPixels': 1e12,
    'scale': 30,
    }

    task = ee.batch.Export.image.toCloudStorage(**params)
    task.start()

In [19]:
# Check task status
!earthengine task list --status RUNNING

In [26]:
!earthengine task list

VE4FBE247HDC56TBMVZHHRJZ  Export.image  toc_1996  RUNNING    ---
2WWOJMVYLIQFV6G7OYQ5TY5M  Export.image  toc_1996  FAILED     Export too large: specified 1036802880000 pixels (max: 100000000). Specify higher maxPixels value if you intend to export a large area.
KF7ISWKSLQM7QANWM2KE4FNN  Export.image  toc_2016  FAILED     Internal error.
FL4NEFRGO3KFYVRG45DBBW2Y  Export.image  toc_2015  FAILED     Execution failed; out of memory.
6OBTPMJVNIKUY3OEX2SNUEW4  Export.image  toc_2010  FAILED     Internal error.
5XVUYEJVRHDS36QJO7KCDTRD  Export.image  toc_2009  FAILED     Internal error.
DP6J7XH5ZECTBWDGHMVGY7DU  Export.image  toc_2008  FAILED     Internal error.
EFN6YM5AOM7MSNOYVJFBZO6V  Export.image  toc_2007  FAILED     Internal error.
L3YACVDFEAFWWVP2RRD4KN5H  Export.image  toc_2000  FAILED     Internal error.
T5WNHDS3WZPX5VPEW5723Y4S  Export.image  toc_1996  FAILED     Internal error.
FT2MLLBM2PHBHAOVFZSUTNSQ  Export.image  toc_1996  FAILED     Export too large: specified 1036802880000 pi

### Step 2: Download tiffs (GCS), Unify into 1 file (gdal) and upload (GCS)

#### Download tiffs

## Unify and upload

In [ ]:
from google.cloud import storage

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client.from_service_account_json('../../datasets/service_account.json')
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )
    
def download_blob(bucket_name, prefix, dl_dir):
    """Downloads a file to the bucket."""
     # bucket_name = 'your-bucket-name'
     # prefix = 'your-bucket-directory/'
     # dl_dir = 'your-local-directory/'

    storage_client = storage.Client.from_service_account_json('../../datasets/service_account.json')
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)  # Get list of files

    for blob in blobs:
        filename = blob.name.replace('/', '_') 
        blob.download_to_filename(dl_dir + filename)  # Download

    print(
        "Folder {} downloaded to {}.".format(
            prefix, dl_dir
        )
    )
    

In [39]:
# Set working directories
path_in = "../../datasets/toc_temporary/"
path_out ="../../datasets/gmw_blue_carbon"
os.makedirs(path_out, exist_ok=True)

In [ ]:
%%time
for y in [1996, 2007, 2008, 2009, 2010, 2015, 2016, 2017, 2018, 2019, 2020]:
    download_blob('mangrove_atlas',f'ee_export_tiffs/total_organic_carbon/toc_{y}/toc_{y}',path_in)
    print("\nBuilding VRT\n")
    !gdalbuildvrt {path_out}/gmw_toc_{y}.vrt {path_in}/*.tif
    print("\nBuilding GeoTIFF\n")
    !gdal_translate -co "BLOCKXSIZE=512" -co "BLOCKYSIZE=512" -co "TILED=YES" -co "BIGTIFF=YES" -co "COMPRESS=DEFLATE" {path_out}/gmw_toc_{y}.vrt {path_out}/gmw_toc_{y}.tif
    print("\nCopying files to GCS\n")
    upload_blob('mangrove_atlas', f'{path_out}/gmw_toc_{y}.tif', f'ee_export_tiffs/total_organic_carbon/v2/toc_{y}.tif')